In [1]:
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [15]:
from os import listdir
# files=[ '101016jmatlet201210091',
#         '101016jsolidstatesciences201609005',
#         '101016jmolcata201411015',
#         '101016jjallcom201605264',
#         '101016jjpowsour201512048',
#         '101016jjpowsour201411037',
#         '101002aenm201300174',
#         '101039c4cy00360h',
#         '101039c5tc00196j',
#         '101038srep02426',
#         '101016jjssc200911025',
#         '101016jssc200704044',
#         '101016jnanoen201410008',
#         '101016jmatlet201701142',
#         '101039c5ta08878j'
# ]
files=[ 
        '101016jssc200704044'
]

ann_files= [f for f in listdir('data') if f.endswith(".ann") and f[:-4] in files]

# print(ann_files)
print(len(ann_files))

count = 2127
result = []

testeeee = 0
testeeee2 = 0

# ann_files = ['101016jjpowsour201203049.ann']

for ann_file in ann_files:
    txt_file = f"{ann_file.split('.ann')[0]}.txt"
    print(f"Processing files: {ann_file}, {txt_file}")

    f1 = open(f"data/{ann_file}", "r")
    f2 = open(f"data/{txt_file}", "r")

    tags = []

    f1_lines = f1.readlines()
    for line in f1_lines:
        columns = line.split("\t")
        if(len(columns) > 3):
            raise "ERROR1"
        if(columns[0].startswith("T")):
            # print(columns)

            tag = columns[1].split()
            # print(tag)
            if(len(tag) > 3):
                raise "ERROR2"

            # tags.append(tag[0])
            # tags_indexes.append((int(tag[1]), int(tag[2])))
    
            # words.append([word for word in columns[2].replace("\n", "").split()])

            tags.append({
                'tag': f"{tag[0]}",
                'tags_indexes': (int(tag[1]), int(tag[2])),
                'word': [word for word in columns[2].replace("\n", "").split()]
            })
            # print(words)

    print(tags)

    import nltk
    nltk.download('punkt')
    from nltk.tokenize import TreebankWordTokenizer as twt

    f2_lines = f2.readlines()
    # print(nltk.sent_tokenize(f2_lines[1]))
    testeeee = testeeee + len(f2_lines)
    teste_sentences = [nltk.sent_tokenize(line) for line in f2_lines]
    # print(teste_sentences)
    testeeee2 = testeeee2 + sum([len(line) for line in teste_sentences])
    # print(teste_sentences)
    teste_sentences = [line if len(line)>0 else ' ' for line in teste_sentences]
    # print(teste_sentences)
    teste_sentences = [i for j in teste_sentences for i in j]
    # print(teste_sentences)
    # print([list(twt().span_tokenize(line)) for line in f2_lines])
    # print(len(f2_lines))
    line_lengths = [list(twt().span_tokenize(line))[-1][1] if len(list(twt().span_tokenize(line))) else 0 for line in teste_sentences]
    # print(line_lengths)
    line_end_indexes = [ i+sum(line_lengths[0:k])+k if k!=0 else i for k,i in enumerate(line_lengths)]
    # print(line_end_indexes)
    erase_lines = [k for k, line in enumerate(line_lengths) if line==0]
    # print(erase_lines)
    line_end_indexes = [line for k, line in enumerate(line_end_indexes) if k not in erase_lines]
    # print(line_end_indexes)
    # print(line_end_indexes)
    teste=" ".join(line.strip() for line in teste_sentences)
    # print(teste)
    # print(teste.find("synthesized"))
    # print(teste.find("critic acid"))
    # print(teste.find("ethanol"))
    # print(teste.find("argon"))

    # from nltk.tokenize import word_tokenize, span_tokenize
    # nltk.download('punkt')
    # print(word_tokenize("Hello, my name is Arthur. How are you?"))
    teste2 = list(twt().span_tokenize(teste))
    # print(teste2)
    # teste3 = list(twt().tokenize(teste))
    # print(teste3)
    # teste4 = list(twt().tokenize("Hello, my name is Arthur. How are you?"))
    # print(teste4)

    # print([teste[word_index[0]:word_index[1]] for word_index in teste2])
    print([(teste[word_index[0]:word_index[1]], word_index) for word_index in teste2])

    line_counter = 0

    # print(tags)

    # print(teste2)
    for word_index in teste2:
        # print(line_counter, line_end_indexes, word_index[0])
        if word_index[0] > line_end_indexes[line_counter]:
            count = count+1
            line_counter = line_counter+1

        word = teste[word_index[0]:word_index[1]]
        if('"' in word):
            word = word.replace('"', "'")
        if("," in word or "=" in word):
            word = f'"{word}"'

        # print(word)
        # print(word_index[0],word_index[1])
        
        found = False
        for tag in tags:
            if((word_index[0] >= tag["tags_indexes"][0] and word_index[1] <= tag["tags_indexes"][1]) or (word_index[0] >= tag["tags_indexes"][0] and word_index[1]-1 <= tag["tags_indexes"][1] and word[-1] == ".")):
                # print("AEA!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
                # print(word_index)
                # print(word_index[0], word_index[1])
                # print(teste[word_index[0]:word_index[1]])
                # print(tag["tags_indexes"])
                # print(tag["tag"])
                # print(tag["word"])

                tag_prefix = "B-"

                if result[-1]["count"] == count and result[-1]["tag"][2:] == tag["tag"]:
                    tag_prefix = "I-"

                if word[-1] == ".":
                    # result.append(f'Sentence: {count},{word[:-1]},{tag["tag"]}\n')
                    result.append({"count": count, "word": word[:-1], "tag": f'{tag_prefix}{tag["tag"]}'})
                    # result.append(f'Sentence: {count},{word[-1]},"O"\n')
                    result.append({"count": count, "word": word[-1], "tag": "O"})
                else:
                    # result.append(f'Sentence: {count},{word},{tag["tag"]}\n')
                    result.append({"count": count, "word": word, "tag": f'{tag_prefix}{tag["tag"]}'})
                found = True
                break
        if not found:
            # print("!!!!!!!!!!!!")
            if word[-1] == ".":
                # result.append(f'Sentence: {count},{word[:-1]},"O"\n')
                result.append({"count": count, "word": word[:-1], "tag": "O"})
                # result.append(f'Sentence: {count},{word[-1]},"O"\n')
                result.append({"count": count, "word": word[-1], "tag": "O"})
            else:
                # result.append(f'Sentence: {count},{word},"O"\n')
                result.append({"count": count, "word": word, "tag": "O"})
        # print(result[-2:])
    f1.close()
    f2.close()
    count = count + 1

result = [f'Sentence: {line["count"]},{line["word"]},{line["tag"]}\n' for line in result]

# print(result)
new_file = open("validation_data.csv", "w")
new_file.writelines(result)
new_file.close()
print(count)
print(testeeee)
print(testeeee2)

# print(list(twt().span_tokenize("3. Percocet , 5/325 , 1-2 tabs PO q4-6h prn pain .")))



1
Processing files: 101016jssc200704044.ann, 101016jssc200704044.txt
[{'tag': 'Operation', 'tags_indexes': (194, 202), 'word': ['prepared']}, {'tag': 'Meta', 'tags_indexes': (228, 255), 'word': ['solid-state', 'reaction', 'method']}, {'tag': 'Material', 'tags_indexes': (259, 262), 'word': ['air']}, {'tag': 'Operation', 'tags_indexes': (347, 352), 'word': ['mixed']}, {'tag': 'Operation', 'tags_indexes': (402, 410), 'word': ['prefired']}, {'tag': 'Operation', 'tags_indexes': (459, 465), 'word': ['ground']}, {'tag': 'Operation', 'tags_indexes': (467, 477), 'word': ['pelletized']}, {'tag': 'Operation', 'tags_indexes': (483, 491), 'word': ['calcined']}, {'tag': 'Operation', 'tags_indexes': (599, 607), 'word': ['obtained']}, {'tag': 'Operation', 'tags_indexes': (628, 634), 'word': ['ground']}, {'tag': 'Operation', 'tags_indexes': (639, 646), 'word': ['pressed']}, {'tag': 'Operation', 'tags_indexes': (726, 733), 'word': ['reduced']}, {'tag': 'Operation', 'tags_indexes': (842, 850), 'word': ['

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
